In [27]:
import pandas as pd

df_trips = pd.read_csv('Trips.csv')
df_persons = pd.read_csv('Persons.csv')
df_households = pd.read_csv('Households.csv')

# Chuẩn hóa kiểu dữ liệu để tránh lệch kiểu (int vs str)
df_trips['household_id'] = df_trips['household_id'].astype(str)
df_persons['household_id'] = df_persons['household_id'].astype(str)
df_households['household_id'] = df_households['household_id'].astype(str)

# Lấy mảng id ở trip và lọc ở person và household
hh_ids = df_trips['household_id'].dropna().unique()
df_persons_filtered = df_persons[df_persons['household_id'].isin(hh_ids)]
df_households_filtered = df_households[df_households['household_id'].isin(hh_ids)]
df_persons_filtered.to_csv('persons_cleaned.csv', index=False)
df_households_filtered.to_csv('households_cleaned.csv', index=False)


/var/folders/cm/gh503hx51qx6vv971brshjdc0000gn/T/ipykernel_3552/1788851136.py:4: DtypeWarning: Columns (3,4,5,8,9,15,23,25,26,27,28,29,30,31,40,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,68,70,71,76,77,80,81,82,83,84,85,86,87,88,89,90,91,92,94,97,98,99,100) have mixed types. Specify dtype option on import or set low_memory=False.
  df_persons = pd.read_csv('Persons.csv')
/var/folders/cm/gh503hx51qx6vv971brshjdc0000gn/T/ipykernel_3552/1788851136.py:5: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_households = pd.read_csv('Households.csv')


In [28]:
person = df_persons_filtered[['household_id', 'person_id', 'age', 'gender', 'education']]
person.dropna(inplace=True)
map_gender = {
    'Girl/Woman (cisgender or transgender)': 'Female',
    'Boy/Man (cisgender or transgender)': 'Male',
    'Non-binary/Something else fits better': 'Non-binary'
}
person['gender'] = person['gender'].replace(map_gender)
person.to_csv('persons_cleaned.csv', index=False)


/var/folders/cm/gh503hx51qx6vv971brshjdc0000gn/T/ipykernel_3552/1891899725.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person.dropna(inplace=True)
/var/folders/cm/gh503hx51qx6vv971brshjdc0000gn/T/ipykernel_3552/1891899725.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  person['gender'] = person['gender'].replace(map_gender)


In [29]:
person = person[person['gender'] != "Missing Response"]
person = person[person['education'] != "Missing: Skip Logic"]
person.to_csv('persons_cleaned.csv', index=False)

In [30]:
map_age = {
    '75-84 years': '75 years or older',
    '85 years or older': '75 years or older'
}
person['age'] = person['age'].replace(map_age)
person.to_csv('persons_cleaned.csv', index=False)




In [33]:
person = person[person['education'] != "Prefer not to answer"]
person.to_csv('persons_cleaned.csv', index=False)
print(len(person))


824


In [34]:
household = df_households_filtered[['household_id', 'hhincome_broad', 'vehicle_count']]
household.dropna(inplace=True)
household.to_csv('households_cleaned.csv', index=False)

/var/folders/cm/gh503hx51qx6vv971brshjdc0000gn/T/ipykernel_3552/1257654586.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  household.dropna(inplace=True)


In [ ]:
household = household[household['hhincome_broad'] != "Prefer not to answer"]
household.to_csv('households_cleaned.csv', index=False)


In [42]:
hh_ids = household['household_id'].unique()
df_persons_filtered = person[person['household_id'].isin(hh_ids)]
df_trips_filtered = df_trips[df_trips['household_id'].isin(hh_ids)]
df_persons_filtered.to_csv('persons_cleaned.csv', index=False)
df_trips_filtered.to_csv('trips_cleaned.csv', index=False)



In [88]:
import pandas as pd

# Đọc dữ liệu
persons = pd.read_csv('persons_cleaned.csv')
households = pd.read_csv('households_cleaned.csv')

# Chuẩn hóa tên/cột khóa nếu cần
key_persons = 'household_id'
key_households = 'household_id'  # đổi lại nếu tên khác

# Đồng bộ kiểu dữ liệu để tránh lệch kiểu
persons[key_persons] = persons[key_persons].astype(str)
households[key_households] = households[key_households].astype(str)

# Merge: chọn kiểu join (inner/left/right/outer)
merged = persons.merge(
    households,
    left_on=key_persons,
    right_on=key_households,
    how='inner',           # 'left' nếu muốn giữ toàn bộ persons
    suffixes=('_person', '_hh')
)

# Lưu kết quả
merged.to_csv('persons_households_merged1.csv', index=False)
print(merged.shape, 'rows')

(775, 7) rows


In [ ]:
trip = pd.read_csv('trips_cleaned.csv')
trip = trip[['household_id','trip_id', 'person_id', 'mode_class','distance_miles', 'depart_time_hour', 'depart_time_minute', 'depart_time_second', 'dest_purpose_cat']]
trip.dropna(inplace=True)
trip.to_csv('trips_cleaned1.csv', index=False)


In [ ]:
trip = trip[trip['mode_class'] != "Missing Response"]
trip = trip[trip['mode_class'] != "Ride Hail"]
trip = trip[trip['mode_class'] != "Other"]
trip = trip[trip['mode_class'] != "School Bus"]
trip = trip[trip['mode_class'] != "Drive HOV2"]
trip = trip[trip['mode_class'] != "Drive HOV3+"]
trip.to_csv('trips_cleaned1.csv', index=False)



In [63]:
print(trip['mode_class'].unique())
print(trip['dest_purpose_cat'].unique())

['Transit' 'Walk' 'Drive SOV' 'Drive HOV2' 'Drive HOV3+' 'Bike'
 'Micromobility']
['Personal Business/Errand/Appointment' 'Home' 'Work' 'Shopping'
 'Social/Recreation' 'Meal' 'Overnight' 'Escort' 'School' 'Change mode']


In [58]:
trip = trip[trip['dest_purpose_cat'] != "Other"]
trip = trip[trip['dest_purpose_cat'] != "School-related"]
trip = trip[trip['dest_purpose_cat'] != "Work-related"]
trip.to_csv('trips_cleaned1.csv', index=False)

In [71]:
mode_map = {
    "Bike":"Bike/Micromobility",
    "Micromobility":"Bike/Micromobility",
}
trip1['mode_class'] = trip1['mode_class'].replace(mode_map)
trip1.to_csv('trips_cleaned2.csv', index=False)


In [89]:
import pandas as pd

# Đọc dữ liệu
persons = pd.read_csv('persons_households_merged.csv')
trips = pd.read_csv('trips_cleaned2.csv')

# Chuẩn hóa tên/cột khóa nếu cần
key_persons = 'person_id'
key_trips = 'person_id'  # đổi lại nếu tên khác

# Đồng bộ kiểu dữ liệu để tránh lệch kiểu
persons[key_persons] = persons[key_persons].astype(str)
trips[key_trips] = trips[key_trips].astype(str)

# Merge: chọn kiểu join (inner/left/right/outer)
merged = persons.merge(
    trips,
    left_on=key_persons,
    right_on=key_trips,
    how='inner',           # 'left' nếu muốn giữ toàn bộ persons
    suffixes=('_person', '_hh')
)

# Lưu kết quả
merged.to_csv('persons_households_trips_merged.csv', index=False)
print(merged.shape, 'rows')

(3650, 15) rows


In [84]:
print(len(df['person_id'].unique()))

586


In [82]:
import pandas as pd
df=pd.read_csv('data.csv')

def summarize_categorical(df, col, labels=None):
    """
    Thống kê tần suất và phần trăm cho biến phân loại.
    """
    series = df[col].map(labels) if labels else df[col]
    freq = series.value_counts(dropna=False)
    percent = series.value_counts(normalize=True, dropna=False) * 100
    summary = pd.DataFrame({
        "Variable": col,
        "Value": freq.index,
        "Frequency/Mean": freq.values,
        "Percentage/SD": percent.round(2).astype(str) + "%"
    })
    return summary

# Ví dụ: áp dụng cho tất cả các cột của df
summaries = []

for col in df.columns:
    try:
        summaries.append(summarize_categorical(df, col))
    except Exception as e:
        print(f"Bỏ qua cột {col}: {e}")

summary_all = pd.concat(summaries, ignore_index=True)

print(summary_all.head(30))

# Xuất ra file
summary_all.to_csv("summary_all_categoricals.csv", index=False)



               Variable     Value  Frequency/Mean Percentage/SD
0   household_id_person  23220745              82         2.34%
1   household_id_person  23107439              66         1.89%
2   household_id_person  23215575              48         1.37%
3   household_id_person  23072538              45         1.29%
4   household_id_person  23165095              45         1.29%
5   household_id_person  23036633              44         1.26%
6   household_id_person  23109643              39         1.11%
7   household_id_person  23216854              39         1.11%
8   household_id_person  23132795              38         1.09%
9   household_id_person  23177095              38         1.09%
10  household_id_person  23031117              37         1.06%
11  household_id_person  23101273              37         1.06%
12  household_id_person  23136722              36         1.03%
13  household_id_person  23045531              34         0.97%
14  household_id_person  23092634       